# 作業詳解

In [ ]:
import os
import json

from rank_bm25 import BM25Okapi

In [ ]:
# !pip install rank_bm25

In [ ]:
os.chdir("../../")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from src.initialization import credential_init
from src.io.path_definition import get_project_dir

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-2024-05-13", temperature=0)


with open(os.path.join(get_project_dir(), 'tutorial', 'Week-1', 'recipe_train.json'), 'r') as f:
    recipe_train = json.load(f)

In [ ]:
tokenized_corpus = []

for recipe in recipe_train:
    tokenized_corpus.append(recipe['ingredients'])

bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
response_schemas = [
        ResponseSchema(name="used ingredients", description="The actual ingredients used in cooking"),
        ResponseSchema(name="extra ingredients", description="extra ingredients that have to be prepared "),
        ResponseSchema(name="result", description="The dish and cooking recipe in detail")
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

# Define human prompt template

system_prompt = PromptTemplate.from_template("""You are an AI assistant as the best chef in the world. You have a great taste and
cooking skills like Gordon Ramsay. You should be able to come up with dish based on `suggested ingredient`, and tell us what extra ingredients 
to be prepared by comparing the ingredients actually used in the cooking and the `existing ingredient`

The `suggested ingredients` are the ingredients suggested by some recipe. You have the freedom to add or remove ingredients to achieve the goal, 
but try to be as faithful to the `suggested ingredient` as possible. 
""")

system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template='existing ingredients:[{existing_ingredients}]; '
                                       'suggested ingredients: [{suggested_ingredients}]\n; '
                                       'format instruction: {format_instructions}',
                              input_variables=["existing_ingredients", "suggested_ingredients"],
                              partial_variables={"format_instructions": format_instructions}
                              )

human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message,
                                                human_message
                                                ])



In [ ]:
with open(os.path.join(get_project_dir(), 'tutorial', 'Week-1', 'recipe_test.json'), 'r') as f:
    recipe_test = json.load(f)

existing_ingredients = recipe_test[0]['ingredients']

bm25.get_top_n(existing_ingredients, recipe_train, n=3)

In [ ]:
suggested_ingredients = bm25.get_top_n(existing_ingredients, recipe_train, n=3)[0]['ingredients']

In [ ]:
existing_ingredients

In [ ]:
suggested_ingredients

In [ ]:
prompt = chat_prompt.invoke({"existing_ingredients": ", ".join(existing_ingredients), 
                             "suggested_ingredients": ", ".join(suggested_ingredients)})

In [ ]:
prompt

In [ ]:
output = model.invoke(prompt)

In [ ]:
final_output = output_parser.parse(output.content)

In [ ]:
final_output.keys()

In [ ]:
final_output['used ingredients']

In [ ]:
suggested_ingredients

In [ ]:
final_output['extra ingredients']

In [ ]:
final_output['result']

In [ ]:
print(final_output['result'])

In [ ]:
translated_result = model.invoke(f"Translate the content into traditional Chinese (繁體中文): {final_output['result']}")

In [ ]:
print(translated_result.content)

# Semantic based retrieval

Semantic-based retrieval is a method of finding information that focuses on understanding the meaning behind the words you use. Instead of just matching exact words, it looks for the context and concepts in your query. Here's a simple way to understand it:

- 1. Meaning Over Words: Imagine you want to find information about "healthy eating". Traditional search might look for documents with the exact phrase "healthy eating". Semantic-based retrieval, however, understands that terms like "nutritious diet" or "balanced diet" are related and will include those in the results.

- 2. Context Awareness: This method takes into account the context in which words are used. For example, if you search for "apple", a traditional search might give you results about the fruit and the tech company. Semantic-based retrieval uses context to determine whether you’re likely asking about a fruit or a tech product.

- 3. Natural Language Understanding: It works more like how humans understand language. When you ask a question, it tries to grasp the intent behind your query and finds relevant information accordingly.

- 4. Better Results: By focusing on the meaning and context, semantic-based retrieval can provide more accurate and relevant results. This means you spend less time sifting through unrelated information.


語義檢索是一種尋找信息的方法，它重點在於理解你使用的詞語背後的意思。與其僅僅匹配精確的詞語，它會尋找你查詢中的上下文和概念。以下是一種簡單的理解方式：

- 1. 重點在於意思：想像一下你想找關於“健康飲食”的信息。傳統搜索可能會尋找包含“健康飲食”這個精確詞語的文檔。而語義檢索則會理解“營養均衡的飲食”或“均衡飲食”等相關詞語，並將它們包含在結果中。

- 2. 上下文感知：這種方法會考慮詞語使用的上下文。例如，如果你搜索“蘋果”，傳統搜索可能會給你關於水果和科技公司的結果。語義檢索則會使用上下文來判斷你更可能是在詢問水果還是科技產品。

- 3. 自然語言理解：它更像人類理解語言的方式。當你提出問題時，它會嘗試理解你查詢背後的意圖，並相應地找到相關信息。

- 4. 更好的結果：通過重點關注意思和上下文，語義檢索可以提供更準確和相關的結果。這意味著你可以減少篩選無關信息的時間。

In [ ]:
# !pip install sentence-transformers

In [ ]:
from langchain.docstore.document import Document

documents = []

for recipe in recipe_train:
    document = Document(page_content=", ".join(recipe['ingredients']),
                        metadata={"cuisine": recipe['cuisine'],
                                  "id": recipe['id']})
    documents.append(document)

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings


# https://platform.openai.com/docs/guides/embeddings/what-are-embeddings

# A list of embedding models you can choose 
# https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

### 1. Creating Embeddings (創建嵌入):

- HuggingFaceEmbeddings is used to create embeddings (vector representations) for text data.
- The model all-MiniLM-L6-v2 from Hugging Face is specified to generate these embeddings. This model converts text into numerical vectors that capture the semantic meaning of the text.

- 使用 HuggingFaceEmbeddings 創建文本數據的嵌入（向量表示）。
- 指定 Hugging Face 的模型 all-MiniLM-L6-v2 來生成這些嵌入。此模型將文本轉換為數字向量，這些向量捕捉文本的語義。

In [ ]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

### 2. Initializing Vector Store (初始化向量存儲):

- Chroma.from_documents is used to create a vector store from a subset of documents.
- The first 500 documents from the documents list are selected for this operation.
- The embedding parameter is set to the previously created embeddings (HuggingFaceEmbeddings).

- 使用 Chroma.from_documents 從一部分文檔創建一個向量存儲。
- 選擇 documents 列表中的前 500 個文檔來進行此操作。
- embedding 參數設置為先前創建的嵌入（HuggingFaceEmbeddings）。

In [ ]:
vectorstore = Chroma.from_documents(documents[:500], embedding=embedding)

### 3. Creating a Retriever (創建檢索器):

- The as_retriever method is called on the vectorstore object to create a retriever.
- This retriever is configured to use "similarity" as the search type, meaning it will find documents that are similar to a given query based on their vector embeddings.

- 在 vectorstore 對象上調用 as_retriever 方法來創建一個檢索器。
- 這個檢索器配置為使用“相似性”作為搜索類型，這意味著它將根據文檔的向量嵌入找到與給定查詢相似的文檔。

### 4. Setting Search Parameters (設置搜索參數):

- The search_kwargs argument is used to pass additional parameters to the search function.
- In this case, {'k': 5} is specified, which means the retriever will return the top 5 most similar documents for each query.

- 使用 search_kwargs 參數來傳遞額外的搜索功能參數。
- 在這裡，指定了 {'k': 5}，這意味著檢索器將返回每個查詢最相似的前 5 個文檔。

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity",
                                     search_kwargs={'k': 5})

In [ ]:
with open(os.path.join(get_project_dir(), 'tutorial', 'Week-1', 'recipe_test.json'), 'r') as f:
    recipe_test = json.load(f)

In [ ]:
query = ", ".join(recipe_test[0]['ingredients'])

In [ ]:
query

In [ ]:
retriever.invoke(query)

## Three search types:

### 1. similarity (default)

- This search type finds documents that are most similar to your query. It looks at the meaning of the words you used and matches documents that have similar meanings. Think of it like finding articles or documents that closely relate to the topic you're interested in.

- 這種搜索類型找到與你的查詢最相似的文檔。它會看你使用詞語的意思，並匹配具有相似意思的文檔。可以把它想像成找到與你感興趣的主題密切相關的文章或文檔。

### 2. MMR, Maximum Marginal Relevance (MMR, 最大邊際相關性):

- This method balances finding documents that are similar to your query while also ensuring that the results are diverse. It's like asking for a variety of opinions on a topic so you don't get too much of the same thing. It helps avoid redundancy in the search results.

- 這種方法在找到與你的查詢相似的文檔的同時，也確保結果是多樣的。這就像是在一個主題上尋求多種意見，避免得到過多相同的東西。它有助於避免搜索結果的冗餘。

### 3. similarity_score_threshold (相似性分數閾值):

- This search type sets a minimum similarity score that documents must meet to be considered relevant. Only documents that are very close to your query in terms of meaning will be included. It ensures that the results are highly relevant and filters out less related information.

- 這種搜索類型設置一個最小相似性分數，只有達到這個分數的文檔才會被認為是相關的。只有那些在意思上與你的查詢非常接近的文檔才會被包含進來。它確保結果高度相關，並過濾掉不太相關的信息。

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://miro.medium.com/v2/resize:fit:720/format:webp/1*c0c19i2tPSWZaHwQ7cVMrg.png")

In [ ]:
"""
cosine similarity

https://api.python.langchain.com/en/latest/_modules/langchain_core/vectorstores.html

elif search_type == "similarity_score_threshold":
    docs_and_similarities = self.similarity_search_with_relevance_scores(
        query, **kwargs
    )
    return [doc for doc, _ in docs_and_similarities]

in subclass.
Return docs and relevance scores in the range [0, 1].

0 is dissimilar, 1 is most similar.
"""

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5, "k":10}
)

In [ ]:
query = ", ".join(recipe_test[0]['ingredients'])

In [ ]:
retriever.invoke(query)

### How to get the scores of the documents?

In [ ]:
vectorstore.similarity_search_with_score(query)

In [ ]:
vectorstore._similarity_search_with_relevance_scores(query)

In [ ]:
vectorstore._select_relevance_score_fn?

In [ ]:
vectorstore.similarity_search_with_score?

### How to leverage the metadata?

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5, "filter": {'cuisine': {'$eq':'mexican'}}}
)

In [ ]:
retriever.invoke(query)

In [ ]:
# What if we have more than one condition?

# template

# filter = {'$and': [{'brand': {'$eq': brand}},  {'category': {'$eq': category}}}]# {
# "filter": filter

# greater than: '$gt' 
# less than: '$lt}

In [ ]:
retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k': 8, 'fetch_k': 50, 'lambda_mult': 0.1})

# **** 預計第一個小時結束 ****

# LangChain Expression Language (LCEL)

Previously, several steps are required to generate the desired result:
1. Create prompt
2. feed the prompt to model
3. parse the result

Can we achieve the result in one step?

### 食譜 - LCEL

In [ ]:
response_schemas = [
        ResponseSchema(name="used ingredients", description="The actual ingredients used in cooking"),
        ResponseSchema(name="extra ingredients", description="extra ingredients that have to be prepared "),
        ResponseSchema(name="result", description="The dish and cooking recipe in detail")
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

# Define human prompt template

system_prompt = PromptTemplate.from_template("""You are an AI assistant as the best chef in the world. You have a great taste and
cooking skills like Gordon Ramsay. You should be able to come up with dish based on `suggested ingredient`, and tell us what extra ingredients to be prepared by 
comparing the ingredients actually used in the cooking and the `existing ingredient`

The `suggested ingredients` are the ingredients suggested by some recipe. You have the freedom to add or remove ingredients to achieve the goal, but try to be as 
faithful to the `suggested ingredient` as possible. 
""")

system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template='existing ingredients:[{existing_ingredients}]; '
                                       'suggested ingredients: [{suggested_ingredients}]\n; '
                                       'format instruction: {format_instructions}',
                              input_variables=["existing_ingredients", "suggested_ingredients"],
                              partial_variables={"format_instructions": format_instructions}
                              )

human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message,
                                                human_message
                                                ])

In [ ]:
chain = chat_prompt|model|output_parser

In [ ]:
chain.invoke({"existing_ingredients": ", ".join(existing_ingredients), "suggested_ingredients": ", ".join(suggested_ingredients)})

## Minimal Example

### 1. Creating a Prompt Template (創建提示模板):

- ChatPromptTemplate.from_template is used to create a prompt template. This template is a string that includes a placeholder {topic}.
- The template specifies the instruction: "tell me a short joke about {topic}".
- 使用 ChatPromptTemplate.from_template 創建一個提示模板。這個模板是一個包含佔位符 {topic} 的字符串。
- 模板指定了指令：“tell me a short joke about {topic}”（給我講一個關於{topic}的簡短笑話）。

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

### 2. Setting Up the Chain (設置鏈條):

- chain = prompt | model sets up a chain where the prompt is connected to a model. This means that the model will process the prompt to generate a response.
- The | operator is used to combine the prompt and the model into a single chain.
- chain = prompt | model 設置了一個鏈條，其中提示連接到模型。這意味著模型將處理該提示來生成回應。
- | 運算符用於將提示和模型組合成一個鏈條。

In [ ]:
chain = prompt | model

### 3. Getting the Joke (獲取笑話):

- The result of chain.invoke({"topic": "ice cream"}) is stored in the variable joke.
- This variable now contains the generated joke about ice cream.
- chain.invoke({"topic": "ice cream"}) 的結果存儲在變量 joke 中。
- 這個變量現在包含生成的關於冰淇淋的笑話。

In [ ]:
joke = chain.invoke({"topic": "ice cream"})

### 1. Importing StrOutputParser (導入 StrOutputParser):

- The code imports StrOutputParser from the langchain_core.output_parsers module. This class is used to parse the output of the model into a string format.
- 代碼從 langchain_core.output_parsers 模塊導入 StrOutputParser。這個類用於將模型的輸出解析為字符串格式。

### 2. Creating an Output Parser:

- An instance of StrOutputParser is created and assigned to the variable output_parser.
- This parser will be used to process the raw output from the model and convert it into a readable string format.
- 創建一個 StrOutputParser 的實例，並將其賦值給變量 output_parser。
- 這個解析器將用於處理來自模型的原始輸出，並將其轉換為可讀的字符串格式。

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

In [ ]:
Image(filename= "tutorial/Week-2/lcel pipeline.png")

## 範例操作

### Coercion

In [ ]:
joke_chain = prompt | model | output_parser

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_chain = {"joke": chain} | analysis_prompt | model | output_parser

In [ ]:
composed_chain.invoke({"topic": "ice cream"})

1. chain 執行結果，將結果放進'joke' 這個 key 裡
2. {"joke": content} 被送進analysis_prompt 中，等價於 analysis_prompt.invoke({"joke": content})
3. model 接收 analysis_prompt 產生的結果
4. output_parser 處理結果

## Parallelize steps

In [ ]:
from langchain_core.runnables import RunnableParallel

joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

In [ ]:
type(joke_chain)

In [ ]:
%%timeit

joke_chain.invoke({"topic": "bear"})

In [ ]:
%%timeit

poem_chain.invoke({"topic": "bear"})

In [ ]:
%%timeit

map_chain.invoke({"topic": "bear"})

RunnableParallel are also useful for running independent processes in parallel, since each Runnable in the map is executed in parallel. For example, we can see our earlier joke_chain, poem_chain and map_chain all have about the same runtime, even though map_chain executes both of the other two.



## Run custom function

In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnableLambda



def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

# chain = (
#     {
#         "a": itemgetter("foo") | RunnableLambda(length_function),
#         "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
#         | RunnableLambda(multiple_length_function),
#     }
#     | prompt
#     | model
# )

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain = (
    {
        "a": itemgetter("foo") | length_function,
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | multiple_length_function,
    }
    | prompt
    | model
)

In [ ]:
chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)


In [ ]:
chain.invoke({"foo": "bar", "bar": "gah"})

How does it work?

- 'bar' -> 'foo', 'foo' ('bar') -> length_function => a = 3
- 'bar' -> 'foo' & 'gah' -> 'bar', 'foo' ('bar') -> 'text1' & 'bar' ('gah') -> 'text2', {'text1': 'bar', 'text2': 'gah'} -> multiple_length_function => b = 9
- {'a':3, 'b': 9} -> prompt -> 'what is 3 + 9'

## Passing data through

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

### Retrieval Example: Step by Step

### 1. Creating a Template (創建模板):

- A template is created that instructs the model to answer a question based only on a provided context. The template looks like this:
- 創建一個模板，指示模型僅基於提供的上下文來回答問題。模板如下

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

### 2. Generating a Prompt (生成提示):

- The ChatPromptTemplate.from_template(template) command uses the template to create a prompt that can later be filled with specific context and a question.
- 使用 ChatPromptTemplate.from_template(template) 命令來創建一個提示，之後可以用特定的上下文和問題來填充。

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

### 3. Formulating a Query (制定查詢):

- A query is created by joining the ingredients from the 6th recipe in recipe_test with commas. This query is used to retrieve relevant information.
- 通過將 recipe_test 中第六個食譜的成分用逗號連接來創建查詢。此查詢用於檢索相關信息。

In [ ]:
query = ", ".join(recipe_test[5]['ingredients'])

### 4. Retrieving Context (檢索上下文):

- The retriever.invoke(query) command uses the query to find the most relevant documents or information. This retrieved information is stored in the context variable.
- 使用 retriever.invoke(query) 命令，通過查詢找到最相關的文檔或信息。這些檢索到的信息存儲在 context 變量中。

In [ ]:
context = retriever.invoke(query)

### 5. Filling the Prompt (填充提示):

- The prompt is filled with the retrieved context and the question using prompt.invoke({"context": context, "question": question}). This creates an input prompt for the model.
- 使用 prompt.invoke({"context": context, "question": question}) 將提示填充檢索到的上下文和問題。這創建了模型的輸入提示。

In [ ]:
question = "Show in all the ingredients."

In [ ]:
prompt_as_input = prompt.invoke({"context": context, "question": question})

### 6. Getting the Model's Response (獲取模型的回應):

- The model is invoked with the filled prompt using model.invoke(prompt_as_input). The model processes the prompt and generates an output.
- 使用 model.invoke(prompt_as_input) 調用模型。模型處理提示並生成輸出。

In [ ]:
output = model.invoke(prompt_as_input)

### 7. Parsing the Output (解析輸出):

- The output from the model is parsed using output_parser.parse(output.content). This ensures the output is in a readable format.
- 使用 output_parser.parse(output.content) 解析模型的輸出。這確保輸出是可讀的格式。

In [ ]:
output_parser.parse(output.content)

In [ ]:
print(output_parser.parse(output.content))

In [ ]:
from langchain_core.runnables import RunnablePassthrough


chain = {"context": itemgetter("query")|retriever, "question":itemgetter("question")} | prompt | model | output_parser

In [ ]:
chain.invoke({"query": query, "question": question})

## Adding values to chain state

- This code snippet demonstrates the use of RunnableParallel in the Langchain library to perform multiple operations in parallel on a given input. Here’s a simple explanation:
- 這段代碼展示了如何在 Langchain 庫中使用 RunnableParallel 來對給定輸入執行多個並行操作。以下是簡單的解釋：

### 1. Creating RunnableParallel (創建 RunnableParallel):

- RunnableParallel is used to execute multiple tasks simultaneously. It takes several runnables (tasks) as arguments, each with a different operation.
- RunnableParallel 用於同時執行多個任務。它將多個可運行的任務作為參數，每個任務執行不同的操作。

### 2. Defining Runnables (定義可運行的任務):

- passed: RunnablePassthrough() is a task that passes the input through without any changes.
- extra: RunnablePassthrough.assign(mult=lambda x: x["num"] * 3) is a task that assigns a new key-value pair mult to the input. The value of mult is calculated by multiplying the input's num value by 3.
- modified: lambda x: x["num"] + 1 is a task that modifies the input by incrementing the num value by 1.
- passed: RunnablePassthrough() 是一個將輸入原樣傳遞的任務，不進行任何更改。
- extra: RunnablePassthrough.assign(mult=lambda x: x["num"] * 3) 是一個為輸入分配新鍵值對 mult 的任務。mult 的值是通過將輸入的 num 值乘以3計算得出的。
- modified: lambda x: x["num"] + 1 是一個將 num 值加1的任務。

In [ ]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

### 3. Invoking RunnableParallel (調用 RunnableParallel):

- runnable.invoke({"num": 1}) executes all the defined tasks on the input {"num": 1} in parallel.
- runnable.invoke({"num": 1}) 對輸入 {"num": 1} 同時執行所有定義的任務。

In [ ]:
runnable.invoke({"num": 1})

## 回家作業

1. 根據食譜 - LCEL, 配合LCEL, 完成從給 材料 -> 中文食譜
2. 根據 retrieval example -> 要求將食材分類 (肉，香料，奶製品，等等)